In [1]:
import numpy as np
# import pandas as pd

In [2]:
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report

In [4]:
import pickle

In [5]:
import pprint
import os
import sys
sys.path.append('../')
from src.load_data import load_data

## Load Data

In [7]:
train_data, valid_data, test_data, metadata = load_data()

In [8]:
number_stopwords = [str(i) for i in range(10001)] + ['0'+str(i) for i in range(100)] + ['000']

In [10]:
scoring = 'f1_macro'
n_jobs=8


In [11]:
tra_sents = np.array([sentence['sentence']
     for article in train_data
     for sentence in article['sentences']])
y_tra = np.array([sentence['label'] for article in train_data for sentence in article['sentences']])

In [12]:
opt_sents = np.array([sentence['sentence']
     for article in (train_data + valid_data)
     for sentence in article['sentences']])

In [13]:
y_opt = np.array([sentence['label'] for article in (train_data + valid_data) for sentence in article['sentences']])

In [14]:
test_sents =  np.array([sentence['sentence']
     for article in test_data
     for sentence in article['sentences']])

In [15]:
y_test = np.array([sentence['label'] for article in test_data for sentence in article['sentences']])

In [16]:
len(tra_sents), len(y_tra), len(test_sents), len(y_test)

(3582, 3582, 441, 441)

# Feature Extraction

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

We can see that best TFIDFVectorizer features are as listed below;
    min_df: 0.001
    max_df: 0.6
    stop_words: num_stopwords

In [18]:
vectorizer = TfidfVectorizer(min_df=0.001, max_df=0.12, stop_words=number_stopwords)
tfidf_vectors = vectorizer.fit_transform(opt_sents)

In [19]:
tra_vectors = vectorizer.transform(tra_sents)
test_vectors = vectorizer.transform(test_sents)

In [20]:
len(vectorizer.vocabulary_)

3010

#### Saving Feature Vectors

import pickle

feature_path = 'Data/features_mindf_0_001_maxdf_0_12_number_stopwords.pickle'

with open(feature_path, 'wb') as file_:
    pickle.dump(tfidf_vectors, file_, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
tfidf_vectors.shape

(3981, 3010)

# Classifier Training
- With hyper-parameter optimization

In [22]:
opt_results = {}
opt_results_path = 'Results/results_050919_tfidf_only_mindf_001_maxdf_0_12_numberstoplwords.pickle'

### Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier


In [24]:
classifier = DecisionTreeClassifier(criterion='gini')

# Decision Tree
params = {
    'max_depth': [None] + [*range(15, 35, 5)],
    'min_samples_split': [*range(50, 200, 20)],
    'min_samples_leaf': [*range(3, 14, 2)],
    'max_features': [None, 'sqrt', 'log2']
}

dt_clf = GridSearchCV(classifier, params, cv=5, refit=False, scoring=scoring, n_jobs=n_jobs)
dt_clf = dt_clf.fit(tfidf_vectors, y_opt)

# print('Best Estimator')
# print(dt_clf.best_estimator_)
print('Best Score')
print(dt_clf.best_score_)
print('Best Params')
print(dt_clf.best_params_)

Best Score
0.757883389796395
Best Params
{'max_depth': None, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 150}


In [25]:
DT = DecisionTreeClassifier(criterion='gini', **dt_clf.best_params_)

In [26]:
DT.fit(tra_vectors, y_tra)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=150,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
y_pred = DT.predict(test_vectors)

In [28]:
opt_results['DecisionTree'] = {}
opt_results['DecisionTree']['GridSearchCV'] = dt_clf
opt_results['DecisionTree']['classif_report'] = classification_report(y_test, y_pred)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90       325
           1       0.79      0.60      0.68       116

   micro avg       0.85      0.85      0.85       441
   macro avg       0.83      0.77      0.79       441
weighted avg       0.85      0.85      0.85       441



In [30]:
with open(opt_results_path, 'wb') as file_:
    pickle.dump(opt_results, file_, protocol=pickle.HIGHEST_PROTOCOL)

### RandomForestClassifier

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
int(3010*0.2)


602

In [ ]:
classifier = RandomForestClassifier(criterion='gini')


params = {
    'n_estimators': [50] + [*range(100, 1000, 100)], 
    'max_depth': [None] + [*range(65, 120, 5)], 
    'min_samples_split': [10, 20, 30, 40, 45, 50, 100, 120, 150],
    'max_features': ['sqrt', 'log2', int(3010*0.1), int(3010*0.2), int(3010*0.3)],
    'bootstrap': [True, False]
}
# params = {
#     'n_estimators': [30, 50, 70, 100, 150], 
#     'max_depth': [None] + [*range(65, 120, 5)], 
#     'min_samples_split': [10, 20, 25, 30, 40, 45, 50, 100, 120, 150],
#     'max_features': ['sqrt', 'log2'],
#     'bootstrap': [True, False]
# }

rf_clf = GridSearchCV(classifier, params, cv=5, refit=False, scoring=scoring, n_jobs=n_jobs)
rf_clf = rf_clf.fit(tfidf_vectors, y_opt)

# print('Best Estimator')
# print(rf_clf.best_estimator_)
print('Best Score')
print(rf_clf.best_score_)
print('Best Params')
print(rf_clf.best_params_)

In [43]:
RF = RandomForestClassifier(criterion='gini', **rf_clf.best_params_, random_state=0)

In [44]:
RF.fit(tra_vectors, y_tra)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=602, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [45]:
y_pred = RF.predict(test_vectors)

In [48]:
opt_results['RandomForest'] = {}
opt_results['RandomForest']['GridSearchCV'] = rf_clf
opt_results['RandomForest']['classif_report'] = classification_report(y_test, y_pred)

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.93      0.89       325
           1       0.75      0.56      0.64       116

   micro avg       0.83      0.83      0.83       441
   macro avg       0.80      0.75      0.77       441
weighted avg       0.83      0.83      0.83       441



In [50]:
with open(opt_results_path, 'wb') as file_:
    pickle.dump(opt_results, file_, protocol=pickle.HIGHEST_PROTOCOL)

###  SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier =  SVC()
params = {
    'kernel': ('linear', 'poly', 'rbf', 'sigmoid'), 
    'C': [0.025, 0.25, 0.5, 1, 2, 3, 4, 5, 6, 7],
}

svc_clf = GridSearchCV(classifier, params, cv=5, refit=False, scoring=scoring, n_jobs=n_jobs)
svc_clf = svc_clf.fit(tfidf_vectors, y_opt)

# print('Best Estimator')
# print(svc_clf.best_estimator_)
print('Best Score')
print(svc_clf.best_score_)
print('Best Params')
print(svc_clf.best_params_)

In [ ]:
SV = SVC(**svc_clf.best_params_, random_state=0)

In [ ]:
SV.fit(tra_vectors, y_tra)

In [ ]:
y_pred = SV.predict(test_vectors)

In [ ]:
opt_results['SVC'] = {}
opt_results['SVC']['GridSearchCV'] = svc_clf
opt_results['SVC']['classif_report'] = classification_report(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
with open(opt_results_path, 'wb') as file_:
    pickle.dump(opt_results, file_, protocol=pickle.HIGHEST_PROTOCOL)

### MLPClassifier

In [62]:
from sklearn.neural_network import MLPClassifier

In [63]:
classifier = MLPClassifier()

params = {
    'hidden_layer_sizes': [(20), (30), (50), (60), (70), (80), (100), (30,10), (50,10), (100,60), (60,20)], 
    'activation': ['relu'], 
    'learning_rate': ['constant', 'adaptive', 'invscaling'],
    'learning_rate_init': [0.1, 0.01, 0.05, 0.001],
    'max_iter': [50, 100, 200, 300, 400]
}
                        
    
#     params = {
#     'hidden_layer_sizes': [(30), (50), (60), (70), (80), (100), (20)], 
#     'activation': ['relu'], 
#     'learning_rate': ['constant', 'adaptive'],
#     'learning_rate_init': [0.01, 0.001, 0.1],
#     'max_iter': [50, 100, 200, 300, 400, 500]
# }
                        
mlp_clf = GridSearchCV(classifier, params, cv=5, refit=False, scoring=scoring, n_jobs=n_jobs)
mlp_clf = mlp_clf.fit(tfidf_vectors, y_opt)



# print('Best Estimator')
# print(mlp_clf.best_estimator_)
print('Best Score')
print(mlp_clf.best_score_)
print('Best Params')
print(mlp_clf.best_params_)

Best Score
0.7418806323618786
Best Params
{'learning_rate_init': 0.05, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'max_iter': 50, 'activation': 'relu'}


In [64]:
ML = MLPClassifier(**mlp_clf.best_params_, random_state=0)

In [65]:
ML.fit(tra_vectors, y_tra)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=100, learning_rate='invscaling',
              learning_rate_init=0.05, max_iter=50, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=0, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [66]:
y_pred = ML.predict(test_vectors)

In [67]:
opt_results['MLP'] = {}
opt_results['MLP']['GridSearchCV'] = mlp_clf
opt_results['MLP']['classif_report'] = classification_report(y_test, y_pred)

In [68]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86       325
           1       0.60      0.48      0.54       116

    accuracy                           0.78       441
   macro avg       0.71      0.68      0.70       441
weighted avg       0.77      0.78      0.77       441



In [69]:
with open(opt_results_path, 'wb') as file_:
    pickle.dump(opt_results, file_, protocol=pickle.HIGHEST_PROTOCOL)

# Read the pickle

with open(opt_results_path, 'rb') as file_:
    opt_results = pickle.load(file_)


In [51]:
opt_results.keys()

dict_keys(['DecisionTree', 'RandomForest'])